<a href="https://colab.research.google.com/github/KarkiAnuj17/Advanced-Calculator/blob/main/Automatic_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/dataset.zip", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

In [9]:

import pandas as pd
import numpy as np
from tqdm import tqdm
import cudf
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag


In [10]:
path="/content/dataset/dataset/test.csv"
df=pd.read_csv(path)
print(df.columns)
print(df.shape)

Index(['id', 'article', 'highlights'], dtype='object')
(11490, 3)


In [11]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
def get_wordnet_pos(treebank_tag):
    # Map POS tag to WordNet POS tag
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text):
    # Split text into sentences
    sentences = sent_tokenize(text)

    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Get English stopwords
    stop_words = set(stopwords.words('english'))

    # Preprocess each sentence
    processed_sentences = []
    for sentence in sentences:
        # Tokenize words in the sentence
        words = word_tokenize(sentence)

        # Get POS tags
        pos_tags = pos_tag(words)

        # Lemmatize and remove stopwords
        processed_words = []
        for word, pos in pos_tags:
            if word.lower() not in stop_words and word.isalnum():
                lemmatized_word = lemmatizer.lemmatize(word.lower(), get_wordnet_pos(pos))
                processed_words.append(lemmatized_word)

        # Append processed sentence
        processed_sentences.append(processed_words)

    return processed_sentences

def preprocess_csv(file_path, text_column):
    # Read the CSV file into a cuDF DataFrame
    df = cudf.read_csv(file_path)

    # Convert text column to a pandas Series for processing with NLTK
    text_series = df[text_column].to_pandas()

    # Apply preprocessing to the text column with progress bar
    processed_text = text_series.progress_apply(preprocess_text)

    # Convert the processed text back to a cuDF DataFrame
    df['processed_text'] = cudf.from_pandas(processed_text)

    return df

# Enable the tqdm progress_apply
tqdm.pandas()

file_path = '/content/dataset/dataset/test.csv'
text_column = 'article'
processed_df = preprocess_csv(file_path, text_column)

# Display the processed DataFrame
print(processed_df.head())

100%|██████████| 11490/11490 [09:07<00:00, 20.97it/s]


                                         id  \
0  92c514c913c0bdfe25341af9fd72b29db544099b   
1  2003841c7dc0e7c5b1a248f9cd536d727f27a45a   
2  91b7d2311527f5c2b63a65ca98d21d9c92485149   
3  caabf9cbdf96eb1410295a673e953d304391bfbb   
4  3da746a7d9afcaa659088c8366ef6347fe6b53ea   

                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                          highlights  \
0  Experts question if  packed out planes are put...   
1  Drunk teenage boy climbed into lion enclosure ...   
2  Nottingham Forest are close to extending Dougi...   
3  Fiorentina goalkeeper Neto has been linked wit...   
4  Tell-all interview with the reality TV star, 6...   

                                      proce

In [25]:
import cupy as cp
from collections import Counter
from tqdm import tqdm

# Vectorize sentences using a Bag-of-Words approach with CuPy
def vectorize_sentences(sentences):
    # Create vocabulary (unique words across all sentences)
    vocabulary = sorted(set(word for sentence in sentences for word in sentence))
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}

    # Create term-document matrix using CuPy
    term_doc_matrix = cp.zeros((len(sentences), len(word_to_index)))

    for i, sentence in enumerate(sentences):
        word_counts = Counter(sentence)
        for word, count in word_counts.items():
            if word in word_to_index:
                term_doc_matrix[i, word_to_index[word]] = count
    return term_doc_matrix

# Compute the cosine similarity matrix using CuPy
def cosine_similarity_matrix(matrix):
    # Matrix should be a CuPy array
    matrix = cp.array(matrix)
    dot_product = cp.dot(matrix, matrix.T)
    norm = cp.linalg.norm(matrix, axis=1)
    similarity_matrix = dot_product / (cp.outer(norm, norm) + 1e-10)  # Prevent division by zero
    return similarity_matrix

# Process each article and compute sentence similarity matrices
def compute_similarity_for_articles(df, text_column='processed_text'):
    text_series = df[text_column].to_pandas()

    # List to hold similarity matrices for each article
    similarity_matrices = []

    # Iterate over each article
    for index, processed_text in tqdm(text_series.items(), total=len(text_series), desc="Processing Articles"):
        processed_sentences = [' '.join(sentence) for sentence in processed_text]

        # Vectorize sentences
        term_doc_matrix = vectorize_sentences(processed_text)

        # Compute similarity matrix
        similarity_matrix = cosine_similarity_matrix(term_doc_matrix)

        # Convert to CPU if needed for output (optional if needed)
        similarity_matrix_cpu = cp.asnumpy(similarity_matrix)

        # Store the similarity matrix for the article
        similarity_matrices.append(similarity_matrix_cpu)

    return similarity_matrices

# Compute and store similarity matrices for each article
final_similarity_matrices = compute_similarity_for_articles(processed_df, text_column='processed_text')

# Example output for the first article
print("Final Cosine Similarity Matrix for the First Article:\n", final_similarity_matrices[0])


Processing Articles: 100%|██████████| 11490/11490 [01:26<00:00, 133.43it/s]


Final Cosine Similarity Matrix for the First Article:
 [[1.         0.09128709 0.         0.1        0.06454972 0.
  0.         0.16329932 0.09534626 0.0766965  0.08451543 0.16903085
  0.21764288 0.         0.048795   0.11952286]
 [0.09128709 1.         0.09622504 0.18257419 0.05892557 0.
  0.20412415 0.0745356  0.08703883 0.         0.15430335 0.07715167
  0.         0.         0.         0.        ]
 [0.         0.09622504 1.         0.63245553 0.13608276 0.19245009
  0.         0.0860663  0.         0.         0.         0.
  0.         0.         0.05143445 0.        ]
 [0.1        0.18257419 0.63245553 1.         0.12909944 0.09128709
  0.         0.16329932 0.09534626 0.         0.08451543 0.08451543
  0.         0.         0.048795   0.        ]
 [0.06454972 0.05892557 0.13608276 0.12909944 1.         0.29462783
  0.         0.10540926 0.12309149 0.         0.10910895 0.10910895
  0.         0.         0.03149704 0.        ]
 [0.         0.         0.19245009 0.09128709 0.294627